In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from warnings import filterwarnings
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.model_selection import train_test_split
from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf


filterwarnings('ignore')

In [2]:
def data_preprocess(data_path, labels_path=None):
    features = ['reanalysis_specific_humidity_g_per_kg', 
                 'reanalysis_dew_point_temp_k', 
                 'station_avg_temp_c',
                'precipitation_amt_mm',
                'ndvi_ne',
                'reanalysis_sat_precip_amt_mm',
                'ndvi_sw',
                'reanalysis_precip_amt_kg_per_m2',
                'station_min_temp_c',
#                 'precipitation_amt_mm', 
                'week_start_date']
    df = pd.read_csv(data_path, index_col=[0, 1, 2])
            
    df['station_avg_temp_c_mv_avg'] = df['station_avg_temp_c'].rolling(window=50).mean()
    df['precipitation_amt_mm_mv_avg'] = df['precipitation_amt_mm'].rolling(window=50).mean()
    features.append('station_avg_temp_c_mv_avg')
    features.append('precipitation_amt_mm_mv_avg') 
    
    df['reanalysis_sat_precip_amt_mm'] =  df['reanalysis_sat_precip_amt_mm'].shift(-20)
    
    df['ndvi_ne_avg'] = df['ndvi_ne'].rolling(window=10).mean()
    features.append('ndvi_ne_avg')
    
    df['ndvi_sw_avg'] = df['ndvi_sw'].rolling(window=30).mean().shift(-10)
    features.append('ndvi_sw_avg')
    
    
    df['reanalysis_precip_amt_kg_per_m2_avg'] = df['reanalysis_precip_amt_kg_per_m2'].rolling(window=50).mean()
    features.append('reanalysis_precip_amt_kg_per_m2_avg')
    
    
    
    df['reanalysis_specific_humidity_g_per_kg_avg'] = df['reanalysis_specific_humidity_g_per_kg'].rolling(window=50).mean()
    features.append('reanalysis_specific_humidity_g_per_kg_avg')
    
    
    df['reanalysis_dew_point_temp_k_avg'] = df['reanalysis_dew_point_temp_k'].rolling(window=35).mean()
    features.append('reanalysis_dew_point_temp_k_avg')

    
    df.fillna(method='ffill', inplace=True)
    df = df.fillna(df.mean())
    
    df['week_start_date'] = pd.to_datetime(df['week_start_date'])
    for i in range(1,5):
        df['quarter_' + str(i)] = df['week_start_date'].apply(lambda date: 1 if (
            ((i-1)*3<date.month) and (date.month <= i * 3)) else 0)
        features.append('quarter_' + str(i))
    
    df = df.drop(['week_start_date'], axis=1)
    features.remove('week_start_date')
    df = df[features]    
    sj_label = None
    iq_label = None
    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path, index_col=[0, 1, 2]).loc[df.index]
        sj_label = pd.DataFrame(labels.loc['sj'])
        iq_label = pd.DataFrame(labels.loc['iq'])

    sj = pd.DataFrame(df.loc['sj'])
    iq = pd.DataFrame(df.loc['iq'])
    
    
    return sj, iq, sj_label, iq_label

In [3]:
sj_train, iq_train, sj_label, iq_label = data_preprocess('./data/train_features.csv', './data/train_labels.csv')

In [4]:
sj_train_X, sj_test_X, sj_train_y, sj_test_y = train_test_split(sj_train, sj_label['total_cases'], test_size=0.1, random_state=0, shuffle=False)

iq_train_X, iq_test_X, iq_train_y, iq_test_y = train_test_split(iq_train, iq_label['total_cases'], test_size=0.25, random_state=0, shuffle=False)
sj_train_X['total_cases'] = sj_train_y
sj_test_X['total_cases'] = sj_test_y

iq_train_X['total_cases'] = iq_train_y
iq_test_X['total_cases'] = iq_test_y

In [5]:


def find_best_model(train, test):
    # Step 1: specify the form of the model
    model_formula = "total_cases ~ 1 + " \
                    "reanalysis_specific_humidity_g_per_kg + " \
                    "reanalysis_dew_point_temp_k + " \
                    "station_avg_temp_c + " \
                    "precipitation_amt_mm + " \
    "station_avg_temp_c_mv_avg + " \
    "precipitation_amt_mm_mv_avg " \
    
    
    grid = 10 ** np.arange(-8, -3, dtype=np.float64)
                    
    best_alpha = []
    best_score = 1000
        
    # Step 2: Find the best hyper parameter, alpha
    for alpha in grid:
        model = smf.glm(formula=model_formula,
                        data=train,
                        family=sm.families.NegativeBinomial(alpha=alpha))

        results = model.fit()
        predictions = results.predict(test).astype(int)
        score = eval_measures.meanabs(predictions, test.total_cases)

        if score < best_score:
            best_alpha = alpha
            best_score = score

    print('best alpha = ', best_alpha)
    print('best score = ', best_score)
            
    # Step 3: refit on entire dataset
    full_dataset = pd.concat([train, test])
    model = smf.glm(formula=model_formula,
                    data=full_dataset,
                    family=sm.families.NegativeBinomial(alpha=best_alpha))

    fitted_model = model.fit()
    return fitted_model

sj_model = find_best_model(sj_train_X, sj_test_X)
iq_model = find_best_model(iq_train_X, iq_test_X)



best alpha =  1e-08
best score =  14.361702127659575
best alpha =  1e-08
best score =  59.738461538461536


In [6]:

sj_test, iq_test, sj_test_label, iq_test_label = data_preprocess('./data/dengue_features_test.csv')

sj_predictions = sj_model.predict(sj_test).astype(int)
iq_predictions = iq_model.predict(iq_test).astype(int)

submission_file = pd.read_csv("./data/submission .csv", index_col=[0, 1, 2])

submission_file.total_cases = np.concatenate([sj_predictions, iq_predictions])
submission_file.to_csv("./results/submission_latest_5.csv")